In [8]:
module NestQuant3;
    input Vector#(8, Bit#(4)) coords;
    Reg#(Vector#(8, Bit#(6))) v(replicate(0));
    Reg#(Bit#(1)) flip1(0);
    Reg#(Bit#(1)) flip2(0);
    Reg#(Bit#(8)) dist0(0);
    Reg#(Bit#(8)) dsum(0);
    Reg#(Vector#(8, Bit#(6))) v1(replicate(0));
    Reg#(Vector#(8, Bit#(6))) out(replicate(0));

    rule do_work;
        Vector#(8, Bit#(6)) p;
        p[0] = {coords[0], 2'b00};
        for (Integer i = 1; i < 7; i = i + 1)
            p[i] = {1'b0, coords[i], 1'b0};
        p[7] = {2'b00, coords[7]};

        Vector#(8, Bit#(6)) _v;
        _v[0] = ((p[0] + p[1]) + (p[2] + p[3])) + ((p[4] + p[5]) + (p[6] + p[7]));
        for (Integer i = 1; i < 7; i = i + 1)
            _v[i] = p[i] + p[7];
        _v[7] = p[7];
        v <= _v;


        Vector#(8, Bit#(5)) anti;
        for (Integer i = 0; i < 8; i = i + 1)
            anti[i] = 5'b10000 - ({1'b0, v[i][3:0]});

        Vector#(8, Bit#(8)) ddist;
        for (Integer i = 0; i < 8; i = i + 1)
            ddist[i] = {3'b000, unpack(v[i][4]) ? {1'b0, v[i][3:0]} : anti[i]};

        Bit#(1) _flip2 = ((v[0][5] ^ v[1][5]) ^ (v[2][5] ^ v[3][5])) ^ ((v[4][5] ^ v[5][5]) ^ (v[6][5] ^ v[7][5]));
        Bit#(1) xor4 = ((v[0][4] ^ v[1][4]) ^ (v[2][4] ^ v[3][4])) ^ ((v[4][4] ^ v[5][4]) ^ (v[6][4] ^ v[7][4]));

        dsum <= ((ddist[0] + ddist[1]) + (ddist[2] + ddist[3])) + ((ddist[4] + ddist[5]) + (ddist[6] + ddist[7]));
        flip1 <= (xor4 ^ _flip2);
        flip2 <= _flip2;
        dist0 <= ddist[0];
        v1 <= v;

        Bit#(8) fsum = dsum - (unpack(flip1) ? (dist0 << 1) : 0);
        fsum = fsum + (unpack(flip2) ? 32 - (dist0 << 1) : 0);
        Bit#(1) use_x2 = pack(fsum >= 64);

        Bit#(1) flip = unpack(use_x2) ? flip2 : flip1;
        Vector#(8, Bit#(6)) _out;
        Vector#(8, Bit#(1)) sw;

        for (Integer i = 1; i < 8; i = i + 1) begin
            sw[i] = v1[i][4] ^ use_x2;
            _out[i] = {
                sw[i],
                sw[i],
                v1[i][3:0]
            };
        end

        sw[0] = v1[0][4] ^ use_x2;
        _out[0] = {
            sw[0] ^ flip,
            sw[0],
            v1[0][3:0]
        };

        out <= _out;
    endrule

    method Vector#(8, Bit#(6)) get = out;
endmodule


no errors found


In [11]:
module NestQuant2;
    input Vector#(8, Bit#(4)) coords;
    Reg#(Vector#(8, Bit#(6))) v(replicate(0));
    Reg#(Vector#(8, Bit#(6))) out(replicate(0));

    rule do_work;
        Vector#(8, Bit#(6)) p;
        p[0] = {coords[0], 2'b00};
        for (Integer i = 1; i < 7; i = i + 1)
            p[i] = {1'b0, coords[i], 1'b0};
        p[7] = {2'b00, coords[7]};

        Vector#(8, Bit#(6)) _v;
        _v[0] = ((p[0] + p[1]) + (p[2] + p[3])) + ((p[4] + p[5]) + (p[6] + p[7]));
        for (Integer i = 1; i < 7; i = i + 1)
            _v[i] = p[i] + p[7];
        _v[7] = p[7];
        v <= _v;


        Vector#(8, Bit#(5)) anti;
        for (Integer i = 0; i < 8; i = i + 1)
            anti[i] = 5'b10000 - ({1'b0, v[i][3:0]});

        Vector#(8, Bit#(8)) ddist;
        for (Integer i = 0; i < 8; i = i + 1)
            ddist[i] = {3'b000, unpack(v[i][4]) ? {1'b0, v[i][3:0]} : anti[i]};

        Bit#(1) flip2 = ((v[0][5] ^ v[1][5]) ^ (v[2][5] ^ v[3][5])) ^ ((v[4][5] ^ v[5][5]) ^ (v[6][5] ^ v[7][5]));
        Bit#(1) xor4 = ((v[0][4] ^ v[1][4]) ^ (v[2][4] ^ v[3][4])) ^ ((v[4][4] ^ v[5][4]) ^ (v[6][4] ^ v[7][4]));
        Bit#(1) flip1 = flip2 ^ xor4;
        Bit#(8) dsum = ((ddist[0] + ddist[1]) + (ddist[2] + ddist[3])) + ((ddist[4] + ddist[5]) + (ddist[6] + ddist[7]));

        Bit#(8) fsum = dsum - (unpack(flip1) ? (ddist[0] << 1) : 0);
        fsum = fsum + (unpack(flip2) ? 32 - (ddist[0] << 1) : 0);
        Bit#(1) use_x2 = pack(fsum >= 64);

        Bit#(1) flip = unpack(use_x2) ? flip2 : flip1;
        Vector#(8, Bit#(6)) _out;
        Vector#(8, Bit#(1)) sw;

        for (Integer i = 1; i < 8; i = i + 1) begin
            sw[i] = v[i][4] ^ use_x2;
            _out[i] = {
                sw[i],
                sw[i],
                v[i][3:0]
            };
        end

        sw[0] = v[0][4] ^ use_x2;
        _out[0] = {
            sw[0] ^ flip,
            sw[0],
            v[0][3:0]
        };

        out <= _out;
    endrule

    method Vector#(8, Bit#(6)) get = out;
endmodule


no errors found


In [9]:
%%synth NestQuant3 -l extended

Compiling module NestQuant3 
Synthesizing circuit with std cell library = extended, O1, target delay = 1 ps

Gates: 963
Area: 1350.48 um^2
Critical-path delay: 354.59 ps (not including setup time of endpoint flip-flop)

Critical path: ~v[42] -> dsum[7:0][5](DFF_in)
               Gate/port   Fanout        Gate delay (ps)  Cumulative delay (ps) 
               ---------   ------        ---------------  --------------------- 
                  ~v[42]        4                    7.0                    7.0 
                     BUF        8                   25.5                   32.5 
                     BUF        5                   25.9                   58.4 
                   NAND3        3                   19.7                   78.1 
                   NAND3        1                   13.5                   91.6 
                   NAND2        3                   15.7                  107.3 
                    XOR2        1                   29.5                  136.8 
     

In [12]:
%%synth NestQuant2 -l extended

Compiling module NestQuant2 
Synthesizing circuit with std cell library = extended, O1, target delay = 1 ps

Gates: 903
Area: 1117.47 um^2
Critical-path delay: 549.62 ps (not including setup time of endpoint flip-flop)

Critical path: ~v[42] -> out._0[5](DFF_in)
               Gate/port   Fanout        Gate delay (ps)  Cumulative delay (ps) 
               ---------   ------        ---------------  --------------------- 
                  ~v[42]        6                    8.7                    8.7 
                     BUF        8                   27.0                   35.7 
                     INV        5                   14.1                   49.8 
                   NAND2        1                   10.5                   60.3 
                   NAND2        6                   21.8                   82.1 
                   NAND2        2                   14.2                   96.3 
                     INV        1                    5.1                  101.4 
        